### Sentiment Analyse 
#### Ausgangslage
In einem Online Shop können Produkte Reviews in Form von Text geschrieben werden. Wir möchten herausfinden ob der Text positiv oder negativ ist und so die Bewertung für das Produkt erstellen zu können.

#### Aufgabe
Anhand von Produkten von Amazon wollen wir den Sentiment für die Produkte berechnen. Dies tun wir mittels Textanalyse und Supervised Learning Algorithmen wie MaxEnt oder Naive Bayes.

In [ ]:
# Installation der NLTK Library
!pip install nltk

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import nltk.classify.util
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.classify import NaiveBayesClassifier
import numpy as np
import re
import string
import nltk
from IPython.core.display import HTML

#### Laden der Produkt Daten

In [ ]:
csv_data = pd.read_csv(r"1429_1.csv")
csv_data.head()

#### Daten für die Sentiment Analyse extrahieren und null values analyse

In [ ]:
data = csv_data[['reviews.rating' , 'reviews.text' , 'reviews.title' , 'reviews.username']]
print(data.isnull().sum())
data.head()

#### Sentiment anhand des Ratings setzen
Alle Produkte die ein Rating von 4 und grösser haben sind positiv. Das DataFrame mit einer neuen Spalte ergänzen die den Wert positiv oder negativ hat.

In [ ]:
# Wieder Enfernen, wird vom Teilnehmer gemacht
data["sentiment_rating"] = data["reviews.rating"] >= 4
data["sentiment_rating"] = data["sentiment_rating"].replace([True , False] , ["positive" , "negative"])
data["sentiment_rating"].value_counts().plot.bar()

#### Text für die Sentiment Analyse vorbereiten
Cleanup des Textes implementieren.

In [ ]:
cleanup_re = re.compile('[^a-z]+')
def cleanup_text(sentence):
    # Wieder löschen, müssen die Teilnehmer coden.
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = cleanup_re.sub(' ', sentence).strip()
    return sentence

#### Cleanup des Text für alle Rows durchführen

In [ ]:
sentiment = data[data["reviews.text"].notnull()]
sentiment["text_clean"] = sentiment["reviews.text"].apply(cleanup_text)
sentiment.head()

#### Implementation der Sentiment Analyse mittels NLTK
Implementierung einer einfachen Sentiment Analyse des Textes. Das Sentiment muss mit einer Spalte mit dem dem Text Sentiment ergänzt werden. Dies kann mittels SentimentIntensityAnalyzer https://www.nltk.org/api/nltk.sentiment.html umgesetzt werden. Für den Score des Textes kann die Methode polarity_scores verwendet werden. Ist der Score 0.0 ist der Sentiment neutral, ist der Score grösser als 0.0 ist der Sentiment positiv ansonsten ist er negativ. 

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sentiment_analyzer = SentimentIntensityAnalyzer()

In [ ]:
def set_sentiment(text):
    score = sentiment_analyzer.polarity_scores(text)
    if score["compound"] == 0.0: 
        return "neutral"
    elif score["compound"] > 0.0:
        return "positive"
    else:
        return "negative"

In [ ]:
sentiment["sentiment_text"] = sentiment["text_clean"].apply(set_sentiment)
sentiment.head()
sentiment["sentiment_text"].value_counts().plot.bar()

#### Ausgabe der negativen Kommentare

In [ ]:
negative_summary = list()
for k, v in sentiment.iterrows():
    if v["sentiment_text"] == "negative":
        negative_summary.append({"text": v["text_clean"], "sentiment": v["sentiment_text"]})
        
pd.set_option('display.max_colwidth', -1)      
pd.DataFrame(negative_summary).head(50)